# 1. Environment Setup

In [1]:
import gurobipy as gp
from gurobipy import GRB
import math
import numpy as np
import pandas as pd

In [2]:
demand_type = 'random'

# 2. Data Acquisition

### 2.1 Variables Definition

In [3]:
alloy = ['all_1','all_2','all_3','all_4','all_5']
supplier = ['sup_1','sup_2','sup_3','sup_4','sup_5']
product = ['prod_1','prod_2','prod_3']
month = ['Init', 'Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
I = len(alloy)
J = len(supplier)
Z = len(product)
M = len(month)
q = 0.05
mu = 1

In [4]:
x_name = []
for i in alloy:
    for j in supplier:
        for m in month:
            #value = 'units of '+i+' sourced from '+j+' in '+m
            value = 'order '+i+' | '+j+' | '+m
            x_name.append(value)
            
s_name = []
for m in month:
    for z in product:
        #value = 'units of '+z+' delayed by one month on '+m
        value = 'delay '+z+' | '+m
        s_name.append(value)
        
D_name = []
for m in month:
    for z in product:
        #value = 'units of '+z+' delivered to satisfy demand in '+m
        value = 'deliver '+z+' | '+m
        D_name.append(value)

lambda_1_name = []
for i in alloy:
    for j in supplier:
        for m in month:
            #value = 'if contract upper range exceeded for '+i+' from '+j+' on month '+m
            value = 'exceed '+i+' | '+j+' | '+m
            lambda_1_name.append(value)

lambda_2_name = []
for i in alloy:
    for j in supplier:
        for m in month:
            #value = 'if contract lower range exceeded for '+i+' from '+j+' on month '+m
            value = 'surrender '+i+' | '+j+' | '+m
            lambda_2_name.append(value)

### 2.2 Data Filepath

In [5]:
file_loc = 'Data Templates.xlsx'

### 2.3 Contract Upper Threshold

In [6]:
contract_upper_threshold = pd.read_excel(file_loc,sheet_name='Contract Upper Threshold',header=2, na_values=['NA'], usecols="B:F")
contract_upper_threshold.index=alloy

C:\Users\siraj\anaconda3\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)


In [7]:
h = contract_upper_threshold
contract_upper_threshold

,Supp A,Supp B,Supp C,Supp D,Supp E
all_1,750.000000,375.000000,625.000000,416.666667,333.333333
all_2,5000.000000,833.333333,2750.000000,1666.666667,416.666667
all_3,3750.000000,1250.000000,2916.666667,2104.166667,437.500000
all_4,208.333333,83.333333,187.500000,145.833333,41.666667
all_5,250.000000,95.833333,191.666667,150.000000,43.333333


### 2.4 Contract Lower Threshold

In [8]:
contract_lower_threshold = pd.read_excel(file_loc,sheet_name='Contract Lower Threshold',header=2, na_values=['NA'], usecols="B:F")
contract_lower_threshold.index=alloy

In [9]:
l = contract_lower_threshold
contract_lower_threshold

,Supp A,Supp B,Supp C,Supp D,Supp E
all_1,208.333333,291.666667,333.333333,375.000000,312.500000
all_2,2520.833333,416.666667,833.333333,500.000000,208.333333
all_3,850.000000,833.333333,1250.000000,1270.833333,429.166667
all_4,20.833333,50.000000,41.666667,54.166667,33.333333
all_5,41.666667,58.333333,44.166667,66.666667,34.166667


### 2.5 Contract Penalties

In [10]:
contract_penalties = pd.read_excel(file_loc,sheet_name='Contract Penalties',header=2, na_values=['NA'], usecols="B:F")
contract_penalties.index=['Penalty Fee %']

In [11]:
p = contract_penalties
contract_penalties

,Supp A,Supp B,Supp C,Supp D,Supp E
Penalty Fee %,1,0.6,0.9,0.7,0.44


### 2.6 Annual Contract Limit

In [12]:
annual_contracted_limit= pd.read_excel(file_loc,sheet_name='Annual Contracted Limit',header=2, na_values=['NA'], usecols="B:F")
annual_contracted_limit = annual_contracted_limit.iloc[:5,:]
annual_contracted_limit.index=alloy

In [13]:
k = annual_contracted_limit
annual_contracted_limit

,Supp A,Supp B,Supp C,Supp D,Supp E
all_1,10927.5,6556.5,4371.0,7867.8,9616.2
all_2,67200.0,33600.0,22400.0,44800.0,56000.0
all_3,17600.0,44000.0,35200.0,35200.0,44000.0
all_4,1560.0,1872.0,1768.0,3120.0,2080.0
all_5,1356.0,3616.0,2938.0,1130.0,2260.0


### 2.7 Unit Cost

In [14]:
unit_cost = pd.read_excel(file_loc,sheet_name='Unit Cost',header=2, na_values=['NA'], usecols="B:F")
unit_cost = unit_cost.iloc[:5,:]
unit_cost.index=alloy

In [15]:
c = unit_cost
unit_cost

,Supp A,Supp B,Supp C,Supp D,Supp E
all_1,3300.0,3290.0,3295.0,3293.0,3285.0
all_2,135.0,128.0,133.0,130.0,125.0
all_3,28.0,22.0,26.0,24.0,20.0
all_4,10300.0,10260.0,10285.0,10270.0,10240.0
all_5,2530.0,2450.0,2500.0,2470.0,2430.0


### 2.8 Product Revenue

In [16]:
prod_rev = pd.read_excel(file_loc,sheet_name='Prod Rev',header=2, na_values=['NA'], usecols="B:F")
prod_rev.index=['Revenue ($)']

C:\Users\siraj\anaconda3\lib\site-packages\pandas\io\excel\_base.py:1272: FutureWarning: Defining usecols with out of bounds indices is deprecated and will raise a ParserError in a future version.
  return self._reader.parse(


In [17]:
r = prod_rev
prod_rev

,Prod 1,Prod 2,Prod 3
Revenue ($),185000,180000,270000


### 2.9 Product Recipe

In [18]:
prod_recipe= pd.read_excel(file_loc,sheet_name='Prod Recipe',header=2, na_values=['NA'], usecols="B:D")
prod_recipe = prod_recipe.iloc[:5,:]
prod_recipe.index=alloy

In [19]:
u = prod_recipe
prod_recipe

,Prod 1,Prod 2,Prod 3
all_1,21.855,10.9275,43.71
all_2,67.200,336.0000,268.80
all_3,0.000,308.0000,176.00
all_4,7.280,5.2000,6.24
all_5,5.650,11.3000,2.26


### 2.10 Predicted Demand

In [20]:
predicted_demand= pd.read_csv(demand_type+'/demand.csv', header=2, na_values=['NA'], index_col=0)
predicted_demand.index=month

In [21]:
d = predicted_demand
predicted_demand

,Prod 1,Prod 2,Prod 3
Init,0.0,0.0,0.0
Jan,48.4,11.0,13.2
Feb,44.0,13.2,16.5
Mar,33.0,15.4,16.5
Apr,35.2,11.0,16.5
May,39.6,16.5,22.0
Jun,44.0,27.5,27.5
Jul,39.6,24.2,35.2
Aug,44.0,22.0,30.8
Sep,35.2,19.8,24.2


# 3. Gurobi

In [22]:
model = gp.Model()

Set parameter Username
Academic license - for non-commercial use only - expires 2022-09-25


### 3.1 Decision variables:

$ x_{i,j,m} $ how much of alloy type $ i $ to source from supplier $ j $ in month $ m $

$ s_{m, z} $ how many units of product type $ z $ will be delayed by one month on month $ m $

$ D_{m, z} $ how many units of product type $ z $ were delivered to satisfy demand on month $ m $

$ \lambda^1_{i, j, m} $ a variable indicating how much the order exceeded the upper contract threshold for alloy $i$ from supplier $j$ on month $m$

$ \lambda^2_{i, j, m} $ a variable indicating how much the order fell short of the lower contract threshold for alloy $i$ from supplier $j$ on month $m$

In [23]:
x = model.addVars(I, J, M, vtype=GRB.CONTINUOUS, lb=0, name=x_name)
s = model.addVars(M,Z, vtype=GRB.CONTINUOUS, lb=0, name=s_name)
D = model.addVars(M,Z, vtype=GRB.CONTINUOUS, lb=0, name=D_name)
lambda1 = model.addVars(I, J, M, vtype=GRB.CONTINUOUS, lb=0, name=lambda_1_name)
lambda2 = model.addVars(I, J, M, vtype=GRB.CONTINUOUS, lb=0, name=lambda_2_name)

### 3.2 Objective function

Maximize 

$$
\sum \limits _{m=1} ^{12}\sum \limits _{z=1} ^{3} D_{m, z} r_{z} - \sum \limits _{m=1} ^{12}\sum \limits _{z=1} ^{3} s_{m, z} r_{z}q - \sum \limits _{i=1} ^{5}\sum \limits _{j=1} ^{5}\sum \limits _{m=1} ^{12} c_{i,j}x_{i,j,m} - \sum \limits _{i=1} ^{5}\sum \limits _{j=1} ^{5}\sum \limits _{m=1} ^{12} \lambda^{1}_{i,j,m}p_{j}c_{i,j} - \sum \limits _{i=1} ^{5}\sum \limits _{j=1} ^{5}\sum \limits _{m=1} ^{12} \lambda^{2}_{i,j,m}p_{j}c_{i,j} $$

In [24]:
revenue = sum(D[m,z]*r.iloc[0,z] for m in range(1, M) for z in range(Z))
delay = sum(s[m,z]*r.iloc[0,z]*q for m in range(1, M) for z in range(Z))
cost = sum(c.iloc[i,j]*x[i,j,m] for i in range(I) for j in range(J) for m in range(1, M))
high = sum(lambda1[i,j,m]*p.iloc[0,j]*c.iloc[i, j] for i in range(I) for j in range(J) for m in range(1, M))
low = sum(lambda2[i,j,m]*p.iloc[0,j]*c.iloc[i, j] for i in range(I) for j in range(J) for m in range(1, M))

In [25]:
model.setObjective(revenue-delay-cost-high-low,GRB.MAXIMIZE)

### 3.3 Constraints:

**Capacity**

$ \sum \limits _{m=1} ^{12} x_{i,j,m} \le k_{i,j} $ $ \forall i = 1,...,5$ and $j = 1,...5 $

In [26]:
for i in range(I):
    for j in range(J):
        model.addConstr(sum(x[i,j,m] for m in range(1, M))<=k.iloc[i,j])

**Month 0 Constraints**
    
$ x_{i,j,m=0} = s_{m=0,z} = D_{m=0} = \lambda^1_{i, j, m=0} = \lambda^2_{i, j, m=0} = 0$ $\forall i = 1, ..., 5$, $j = 1, ..., 5 $, and $ z = 1, .., 3$

In [27]:
model.addConstrs(x[i,j,0]==0 for i in range(I) for j in range(J))
model.addConstrs(lambda1[i,j,0]==0 for i in range(I) for j in range(J))
model.addConstrs(lambda2[i,j,0]==0 for i in range(I) for j in range(J))
model.addConstrs(s[0,z]==0 for z in range(Z))
model.addConstrs(D[0,z]==0 for z in range(Z))

{0: <gurobi.Constr *Awaiting Model Update*>,
 1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>}

**Enough Alloys are Sourced to Meet Delivery Requirements (no inventory passed on to next month)**
    
$\sum \limits _{j=1} ^{5} x_{i,j,m} = \sum \limits _{z=1} ^{3} D_{m,z} u_{i,z}  $ $ \forall m = 1,...,12 $ and $i = 1, ..., 5 $

In [28]:
for m in range(1, M):
    for i in range(I):
        model.addConstr(sum(x[i,j,m] for j in range(J))==sum(D[m,z]*u.iloc[i,z] for z in range(Z)))

**Maximum Delay Capacity**

$s_{m, z} \le \mu (d_{m, z}+s_{m-1, z}) $ $ \forall m = 1,...,12 $ and $ z = 1,...,3 $

In [29]:
for m in range(1, M):
    for z in range(Z):
        model.addConstr(s[m,z]<=mu*(d.iloc[m,z]+s[m-1,z]))

**Demand Delivered per Month**

$D_{m, z} = s_{m-1, z} + d_{m, z} - s_{m, z} $ $ \forall m = 1, ..., 12$ and $ z = 1, ..., 3 $

In [30]:
model.addConstrs(s[0,z]==0 for z in range(Z))

for m in range(1,M):
    for z in range(Z):
        model.addConstr(D[m,z] == s[m-1,z] + d.iloc[m,z] - s[m,z])

**Meet All Demand Predicted Within the Year**

$\sum \limits _{m=1}^{12} D_{m,z} = \sum \limits _{m=1}^{12} d_{m, z} $

In [31]:
for z in range(Z):
    model.addConstr(sum(D[m,z] for m in range(1, M)) == sum(d.iloc[m,z] for m in range(1, M)))

**Upper limit penalty**

$  x_{i,j,m} - h_{i,j} \le \lambda^1_{i, j, m} $ $\forall i = 1,...,5 $,  $ j = 1,...,5 $, and $ m = 1,...,12 $

In [32]:
for i in range(I):
    for j in range(J):
        for m in range(M):
            model.addConstr(x[i,j,m]-h.iloc[i,j] <= lambda1[i,j,m])

**Lower limit penalty**

$ l_{i,j} - x_{i,j,m} \le M \lambda^2_{i, j, m}  \forall i = 1,...,5 $, $ j = 1,...,5 $, and $ m = 1,...,12 $ where $ M $ is a very large number

In [33]:
for i in range(I):
    for j in range(J):
        for m in range(1, M):
            model.addConstr(l.iloc[i,j]-x[i,j,m] <= lambda2[i,j,m])

**Non-negativity constraints**

$ x_{i,j,m} \ge 0$ $ \forall i = 1,...,5 $, $ j = 1,...,5 $, and $ m = 1,...,12 $

$ s_{m, z} \ge 0$ $\forall m = 1,...,12 $ and $ z = 1,...,3 $

$ D_{m, z} \ge 0$ $\forall m = 1,...,12 $ and $ z = 1,...,3 $

$ \lambda^1_{i, j, m} \ge 0$  $\forall i = 1,...,5$, $ j=1, ..., 5$, and $ m = 1,...,12 $

$ \lambda^2_{i, j, m} \ge 0$  $\forall i = 1,...,5$, $ j=1, ..., 5$, and $ m = 1,...,12 $

In [34]:
# these constraints are already applied while setting up decision variables

# 4. Result

In [35]:
model.optimize()

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 869 rows, 1053 columns and 2318 nonzeros
Model fingerprint: 0xd91e9e2b
Coefficient statistics:
  Matrix range     [1e+00, 3e+02]
  Objective range  [9e+00, 3e+05]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+01, 7e+04]
Presolve removed 115 rows and 84 columns
Presolve time: 0.01s
Presolved: 754 rows, 969 columns, 2169 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.2802263e+09   9.361643e+04   0.000000e+00      0s
    1481    7.4383432e+06   0.000000e+00   0.000000e+00      0s

Solved in 1481 iterations and 0.03 seconds (0.01 work units)
Optimal objective  7.438343182e+06


In [36]:
for i in range(I):
    orders = pd.DataFrame(columns=supplier, index=month)
    
    for j in range(J):
        for m in range(M):
            
            orders.loc[month[m], supplier[j]] = x[i, j, m].X

    orders.to_csv(demand_type+'/all_'+str(i+1)+'_opt.csv')

In [37]:
costs = pd.Series(dtype='float64')


costs['PO_cost'] = sum(c.iloc[i,j]*x[i,j,m].X for i in range(I) for j in range(J) for m in range(1, M))
costs['delay_cost'] = sum(s[m,z].X*r.iloc[0,z]*q for m in range(1, M) for z in range(Z))
costs['exceed_cost'] = sum(lambda1[i,j,m].X*p.iloc[0,j]*c.iloc[i, j] for i in range(I) for j in range(J) for m in range(1, M))
costs['surrender_cost'] = sum(lambda2[i,j,m].X*p.iloc[0,j]*c.iloc[i, j] for i in range(I) for j in range(J) for m in range(1, M))

costs.to_csv(demand_type+'/opt_cost.csv')

In [38]:
def sen_report(model):
    print('Sensitivity Analysis (SA)\n ObjVal =', model.ObjVal)
    model.printAttr(['X', 'Obj', 'SAObjLow', 'SAObjUp'])
    model.printAttr(['X', 'RC', 'LB', 'SALBLow', 'SALBUp', 'UB', 'SAUBLow', 'SAUBUp'])
    model.printAttr(['Sense', 'Slack', 'Pi', 'RHS', 'SARHSLow', 'SARHSUp']) # Pi = shadow price
    # NOTE: printAttr prints only rows with at least one NON-ZERO value, e.g. model.printAttr('X') prints only non-zero variable values